In [6]:
import MySQLdb
import pandas
import itertools
import numpy as np
import bm25fe
import pickle

In [7]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur = conn.cursor()

sql = '''SELECT bug_id, short_desc, long_desc
FROM bugs_cpdplatform'''

# bugs = pandas.io.sql.read_sql(sql, conn).set_index(['bug_id'])
bugs = pandas.io.sql.read_sql(sql, conn)

In [8]:
with open('train100.txt', 'rb') as f:
    train = pickle.load(f)

examples = []
for item in train:
    examples.append((item['query'], item['rel'], item['irrel'][np.random.randint(len(item['irrel']))]))

In [9]:
from gensim import corpora
from gensim import matutils

# dictionary = corpora.Dictionary(list(bugs_train['text']))
# dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n = 100000)
dictionary = corpora.Dictionary.load_from_text('dictionary.txt')
num_terms = len(dictionary)
print num_terms

bugs['text'] = (bugs['short_desc'] +' '+ bugs['long_desc']).map(lambda x: dictionary.doc2bow(x.split()))
bugs.loc[:,'short_desc'] = bugs['short_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])
bugs.loc[:,'long_desc'] = bugs['long_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])

8647


In [10]:
appearance = np.array(list(bugs['text'].map(lambda x: matutils.corpus2dense([x], num_terms, 1)[:,0]>0)))
df = appearance.sum(0)
idf = np.log(bugs.shape[0]/df)
avgfl = np.array([np.array(list(bugs['short_desc'])).sum(1).mean(), np.array(list(bugs['long_desc'])).sum(1).mean()])

bugs = bugs.set_index(['bug_id'])

In [17]:
params = np.concatenate((np.random.rand(20,10),np.random.rand(30,10)*5,np.random.rand(50,10)*10), axis = 0)

for item in examples:
    for row in params:
        bm = bm25fe.bm25fe(K1=row[0], d_B=(row[1], row[2]), d_W = (row[3], row[4]), K3=row[5], q_B=(row[6], row[7]), q_W=(row[8], row[9]))
        differ = (bm.score(idf, avgfl, [bugs.loc[item[2],'short_desc'], bugs.loc[item[2],'long_desc']], [bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']])-
                  bm.score(idf, avgfl, [bugs.loc[item[1],'short_desc'], bugs.loc[item[1],'long_desc']], [bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']]))
        der = np.exp(differ)/(1+np.exp(differ))*(bm.derivative(idf, avgfl, [bugs.loc[item[2],'short_desc'], bugs.loc[item[2],'long_desc']],[bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']])-
                                                  bm.derivative(idf, avgfl, [bugs.loc[item[1],'short_desc'], bugs.loc[item[1],'long_desc']],[bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']]))
        cost = np.log(1+np.exp(differ))
        
        shift = np.ones(10)*1e-6
        row_shift = row+shift
        bm_shift = bm25fe.bm25fe(K1=row_shift[0], d_B=(row_shift[1], row_shift[2]), d_W = (row_shift[3], row_shift[4]), K3=row_shift[5], q_B=(row_shift[6], row_shift[7]), q_W=(row_shift[8], row_shift[9]))
        differ_shift = (bm_shift.score(idf, avgfl, [bugs.loc[item[2],'short_desc'], bugs.loc[item[2],'long_desc']], [bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']])-
                        bm_shift.score(idf, avgfl, [bugs.loc[item[1],'short_desc'], bugs.loc[item[1],'long_desc']], [bugs.loc[item[0],'short_desc'], bugs.loc[item[0],'long_desc']]))
        cost_shift = np.log(1+np.exp(differ_shift))
        
        error = cost+np.dot(der, shift)-cost_shift
        if abs(error) > 1e-2:
            print item, row, der, error


(720942, 704605, 1408654) [ 6.76005108  1.13377227  0.49804804  2.17796134  0.19876415  7.5756289
  6.19836555  4.30704544  7.03280554  1.80047677] [ -9.88964846e+01   1.13696400e+01  -4.94193990e+02   2.01940794e+01
   3.24427931e+03  -7.01073584e+04   1.43979708e+04  -3.17527510e+05
  -1.81993309e+04   3.66080956e+05] -0.0222061980643
(158778, 158787, 1309631) [ 3.29928177  4.37553387  2.41804467  0.22269837  1.50591021  4.71099279
  1.33939696  3.68647295  4.30058067  4.23027822] [  409332.22229934    -5213.20381081  1006088.18765393   165450.0926741
  -921175.60501498    23767.21844194    13516.87095242    32336.72436085
    -8002.19339628   -18297.28663081] -0.0185919798858
(894915, 894914, 259537) [ 3.84132682  9.26672526  0.01742684  5.00395094  0.65994632  2.9493432
  9.82047118  0.54258627  4.38120586  3.34814264] [ -5.56485646e+04  -7.67955627e+04  -2.41052502e+04   6.54453422e+04
  -1.71783848e+05  -3.50056000e+03  -1.68671664e+03  -9.49677793e+02
   2.55552388e+03  -1.61232

In [16]:
print len(examples)
print len(params)

100
100


In [116]:
# print n
# print len(texts)
# print len(bugset)
# print train
# print bugs_filtered.shape
# print criterion
# print bugset
# print bugs.shape
# print type(criterion1)

In [118]:
# print type(vectors[:n])
# print vectors[0]
# print len(vectorizer.vocabulary_)
# print bug_feature.iloc[0,1]
# print list(vectors_sd)[0]
# print vectors_sd.sum(1).mean()

In [121]:
# bm25fe = reload(bm25fe)